In [105]:
import numpy as np
import pandas as pd
import torch
import pickle as pkl
import warnings
import zipfile
from tqdm import tqdm
import os        
import urllib.request
import gzip
import os.path
import io
from itertools import starmap
from tdc.multi_pred import DDI
from tdc.utils import get_label_map
from tdc.resource import PrimeKG

# Fetch Data

In [93]:
data = {}

## Load Therapeutics Data Commons for DDI

In [181]:
ddi_df = DDI(name = 'DrugBank').get_data()

Found local copy...
Loading...
Done!


In [182]:
data['Drugs'] = pd.DataFrame({'Drug': ddi_df[['Drug1_ID', 'Drug1']].drop_duplicates().set_index('Drug1_ID').to_dict()['Drug1'] | ddi_df[['Drug2_ID', 'Drug2']].drop_duplicates().set_index('Drug2_ID').to_dict()['Drug2']}).rename_axis('Drug_ID')['Drug']
data['DDI'] = ddi_df.set_index(['Drug1_ID', 'Drug2_ID'])[['Y']]
data['DDI Labels'] = pd.Series(get_label_map(name = 'DrugBank', task = 'DDI')).rename_axis('Y').rename('Label')

## Load DrugBank for SMILES

In [ ]:
# download from https://go.drugbank.com/releases/5-1-10/downloads/all-drug-links
zipfile.ZipFile('data/drugbank_all_drug_links.csv.zip').extractall('data')
drugbank = pd.read_csv('data/drug links.csv', index_col=0)

# download from https://go.drugbank.com/releases/5-1-10/downloads/all-structures
zipfile.ZipFile('data/drugbank_all_structures.sdf.zip').extractall('data')
with open('data/structures.sdf') as f:
    col = None
    for line in tqdm(f):
        if line.startswith('> <'):
            col = line[3:-2]
        elif col:
            if col == 'DATABASE_ID':
                key = line[:-1]
            else:
                drugbank.loc[key, col] = line[:-1]
            col = None


In [229]:
drugbank.to_csv('data/drugbank.csv')
data['Drugs'] = pd.Series(data['Drugs'].to_dict() | drugbank['SMILES'].dropna().to_dict())
data['Drugs'].name = 'Drug'
data['Drugs'].index.name = 'Drug_ID'
drugs = set(data['Drugs'].index)
len(drugs)

11597

## Load HGNC

In [92]:
HGNC = pd.read_csv('https://ftp.ebi.ac.uk/pub/databases/genenames/hgnc/tsv/hgnc_complete_set.txt', sep='\t', low_memory=False)

def get_map(from_: str, to: str) -> pd.Series:
    return HGNC[[from_, to]].dropna().set_index(from_)[to]

In [95]:
geneName_2_ccdID = get_map('entrez_id', 'ccds_id')
geneName_2_ccdID.index = geneName_2_ccdID.index.astype(int)
geneName_2_ccdID = geneName_2_ccdID.map(lambda x: x.split('|')[-1])

## Load CCD for Protein Sequence

In [96]:
CCDS_protein_url = 'https://ftp.ncbi.nlm.nih.gov/pub/CCDS/current_human/CCDS_protein.current.faa.gz'
                 # 'https://ftp.ncbi.nlm.nih.gov/pub/CCDS/current_human/CCDS_protein_exons.current.faa.gz'

def fetch_faa(url):
    return gzip.GzipFile(fileobj=io.BytesIO(urllib.request.urlopen(url).read()), mode='rb').read().decode('utf-8')

CCDS_protein_raw = fetch_faa(CCDS_protein_url)

In [97]:
ccd_data = {}
for line in CCDS_protein_raw.splitlines():
    if line.startswith('>'):
        ccd_data[ccd_id:=line[1:].strip().split('|')[0].split('.')[0]] = ''
    else:
        ccd_data[ccd_id] += line
ccdID_2_ccdSeq = pd.Series(ccd_data)

In [118]:
data['Proteins'] = geneID_2_ccdSeq = geneName_2_ccdID.map(ccdID_2_ccdSeq).rename_axis('Protein_ID').rename('Protein').dropna().sort_index()
proteins = set(data['Proteins'].index)
len(proteins)

18929

## Load PrimeKG for PPI and DPI

In [104]:
kg = PrimeKG()

Found local copy...
Loading...


In [230]:
# PPI_df = kg.df.query('relation == "protein_protein"')[['x_id', 'y_id']].astype(int).set_index(['x_id', 'y_id']).rename_axis(['Protein1_ID', 'Protein2_ID'])
data['PPI'] = kg.df.query('relation == "protein_protein"')[['x_id', 'y_id']].astype(int).rename({'x_id': 'Protein1_ID', 'y_id': 'Protein2_ID'}, axis=1)
data['DPI'] = kg.df.query('x_type == "drug" and y_type == "gene/protein"')[['x_id', 'y_id']].astype(({'x_id': str, 'y_id': int})).rename({'x_id': 'Drug_ID', 'y_id': 'Protein_ID'}, axis=1)

# filter out those interactions with protein that are not provided with CCD sequence
data['PPI'] = data['PPI'][data['PPI'].isin(proteins).all(axis=1)].set_index(['Protein1_ID', 'Protein2_ID'])
data['DPI'] = data['DPI'][data['DPI']['Protein_ID'].isin(proteins)]

# filter out those interactions with drugs that are not provided with SMILES
data['DPI'] = data['DPI'][data['DPI']['Drug_ID'].isin(drugs)].set_index(['Drug_ID', 'Protein_ID'])

# drop the proteins that does not occur in any iteractions
data['Proteins'] = data['Proteins'][sorted(set(data['DPI'].reset_index()['Protein_ID']) | set(data['PPI'].reset_index()['Protein1_ID']))]

## Save CSV

In [272]:
data['DDI'].to_csv("../data/csv/DDI.csv")
data['PPI'].to_csv("../data/csv/PPI.csv")
data['DPI'].to_csv("../data/csv/DPI.csv")
data["Drugs"].to_csv("../data/csv/Drugs.csv")
data["Proteins"].to_csv("../data/csv/Proteins.csv")
data['DDI Labels'].to_csv('../data/csv/DDI Labels.csv')

# Generate Features

In [246]:
data = {
    'DDI': pd.read_csv("../data/csv/DDI.csv", index_col=(0,1)),
    'DPI': pd.read_csv("../data/csv/DPI.csv", index_col=(0,1)),
    'PPI': pd.read_csv("../data/csv/PPI.csv", index_col=(0,1)),
    'Proteins': pd.read_csv("../data/csv/Proteins.csv", index_col=0)['Protein'],
    'Drugs': pd.read_csv("../data/csv/Drugs.csv", index_col=0)['Drug'],
    'DDI Labels': pd.read_csv('../data/csv/DDI Labels.csv', index_col=0)['Label']
}

In [132]:
from transformers import AutoTokenizer, AutoModel, EsmTokenizer

ChemBert = {
    'tokenizer': AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM"),
    'model': AutoModel.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
} 

Esm1b = {
    'tokenizer': EsmTokenizer.from_pretrained('facebook/esm-1b', do_lower_case=False),
    'model': AutoModel.from_pretrained("facebook/esm-1b")
}

Some weights of the model checkpoint at DeepChem/ChemBERTa-77M-MLM were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be 

In [133]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ChemBert['model'] = ChemBert['model'].to(device)
Esm1b['model'] = Esm1b['model'].to(device)

In [134]:
def feature_generator(tokenizer, model):
    def get_features(text):
        return model(**dict(starmap(lambda k, v: (k, v.to(device)), tokenizer(text, return_tensors='pt', max_length=512, padding=True, truncation=True).items()))).pooler_output[0].detach().cpu().numpy()
    return get_features

In [247]:
data['DrugFeatures'] = np.array(list(map(feature_generator(**ChemBert), tqdm(data['Drugs']))))

100%|██████████| 11597/11597 [00:55<00:00, 207.60it/s]


In [249]:
data['ProteinFeatures'] = np.array(list(map(feature_generator(**ChemBert), tqdm(data['Proteins']))))

100%|██████████| 17258/17258 [01:28<00:00, 195.16it/s]


## Save NPY

In [252]:
np.save("../data/npy/DrugFeatures.npy", data['DrugFeatures'])
np.save("../data/npy/ProteinFeatures.npy", data['ProteinFeatures'])

# Get Similarity

In [253]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs

def calculate_similarity(smiles1, smiles2):
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)

    try:
        fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, 2, nBits=2048)
        fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, 2, nBits=2048)
    except:
        print(smiles1, smiles2)
        return np.nan
    similarity = DataStructs.TanimotoSimilarity(fp1, fp2)
    return similarity


In [ ]:
def get_similarity(row):
    try:
        return calculate_similarity(
            data['Drugs']['Drug'][row['Drug1_ID']], 
            data['Drugs']['Drug'][row['Drug2_ID']]
        )
    except:
        print(row['Drug1_ID'], row['Drug2_ID'])
data['DDI']['Similarity'] = data['DDI'].apply(get_similarity, axis=1)

# Save

In [271]:
pkl.dump(data, open('../dataset.pkl', 'wb'))

In [256]:
for p1, p2 in data['PPI'].index:
    assert (p2, p1) in data['PPI'].index

# Load

In [273]:
data = {
    'DDI': pd.read_csv("../data/csv/DDI.csv", index_col=(0,1)),
    'DPI': pd.read_csv("../data/csv/DPI.csv", index_col=(0,1)),
    'PPI': pd.read_csv("../data/csv/PPI.csv", index_col=(0,1)),
    'Proteins': pd.read_csv("../data/csv/Proteins.csv", index_col=0)['Protein'],
    'Drugs': pd.read_csv("../data/csv/Drugs.csv", index_col=0)['Drug'],
    'DDI Labels': pd.read_csv('../data/csv/DDI Labels.csv', index_col=0)['Label'],
    'DrugFeatures': np.load("../data/npy/DrugFeatures.npy"),
    'ProteinFeatures': np.load("../data/npy/ProteinFeatures.npy"),
}

In [276]:
data = pkl.load(open('../dataset.pkl', 'rb'))

In [275]:
data.keys()

dict_keys(['DDI', 'DPI', 'PPI', 'Proteins', 'Drugs', 'DDI Labels', 'DrugFeatures', 'ProteinFeatures'])